In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import gaitFunctions
import combineClips
import plotClip
import os
import glob
import seaborn as sns
from scipy import stats
import cv2

def meanstdstr(vals):
    return str(np.round(np.mean(vals),2)) + ' ± ' + str(np.round(np.std(vals),2))

In [ ]:
# plot colors and parameters
axis_fontsize=11
tick_fontsize=9

# colors for step kinematics
stance_color = '#5656ff'
swing_color = '#e1e813'
duty_color = '#4d4d4d'
ipsi_color = 'tab:red'
contra_color = 'tab:green'

# colors for legs
first_pair_color = '#b45eff' # lavender
first_pair_color_alt = '#661f99' # purple
second_pair_color = '#006f2a' # green
second_pair_color_alt = '#02db55' # light green
third_pair_color = '#ffd500' # yellow
third_pair_color_alt = '#fa9805' # orange
rear_leg_color = '#00b6eb'# light blue
rear_leg_color_alt = '#0010eb' # dark blue

# scatter marker size and alpha
marker_size = 10
marker_alpha = 0.7
marker_color = 'k'

# select the folders to include in the analysis

In [3]:
# navigate to directory that contains the data
os.chdir('/Users/iwoods/OneDrive - Ithaca College/2024_movement_disorder_paper/videos_disulfiram/')
# list directories within this directory
dirs = []
for dirname, dirnames, filenames in os.walk('.'):
    # print path to all subdirectories first.
    for subdirname in dirnames:
        folder = os.path.join(dirname, subdirname).split('/')[1]
        if folder not in dirs:
            dirs.append(folder)
dirs = np.sort(np.unique(np.array(dirs)))
dirs = [x for x in dirs if 'Disulfiram' in x]
folders_in_analysis = gaitFunctions.selectMultipleFromList(dirs) 


Choose from this list (separate by commas if multiple choices): 
1: Disulfiram 12Jan24 Inverted
2: Disulfiram 13Dec23 Inverted
3: Disulfiram 16Dec22 Inverted
4: Disulfiram 8Dec22 Inverted
5: xDisulfiram 21Nov22 dissectingscope
6: zDisulfiram 16Nov23 TTclass StudentVids
7: zDisulfiram 21April23 weakdrug
8: select ALL

Which number(s) do you want? 1,2,3,4
You chose: Disulfiram 12Jan24 Inverted and Disulfiram 13Dec23 Inverted and Disulfiram 16Dec22 Inverted and Disulfiram 8Dec22 Inverted


In [ ]:
# in each folder, look for a 'combined' excel file and load (or append) it to a dataframe
# if there is no 'combined' excel file, run combineClips and then load the file

# see how many things are in the 'treatment' column ... may need to fix.